### Install dependencies

In [ ]:
%%bash
# git clone https://github.com/KaiyangZhou/Dassl.pytorch.git
cd Dassl.pytorch/
# Install dependencies
pip install -r requirements.txt
# Install this library (no need to re-build if the source code is modified)
python setup.py develop
cd ..
# Install dependencies of MaPLe
pip install -r requirements.txt

## generate ground truth instance labels

In [44]:
import os
import numpy as np
import torch
import glob

os.makedirs("../../dataset/data/replica_features/train", exist_ok=True)
os.makedirs("../../dataset/data/replica_features/val", exist_ok=True)
os.makedirs("../../dataset/data/replica_features/test", exist_ok=True)
train_test_dict = {
    "office0": "train",
    "office1": "train",
    "office2": "train",
    "office3": "val",
    "office4": "test",
    "room0": "train",
    "room1": "val",
    "room2": "test",
}

gt_masks = glob.glob("../../dataset/OpenYOLO3D/output/replica/replica_ground_truth_masks/*.pt")
# gt_instances = glob.glob("dataset/replica_split/ground_truth/*.txt")
for gt_mask_path in gt_masks:
    gt_mask = torch.load(gt_mask_path)[0]
    gt_instance_id_path = gt_mask_path.replace("OpenYOLO3D/output/replica/replica_ground_truth_masks/", "data/replica_split/ground_truth/").replace(".pt", ".txt")
    gt_instance_id = np.loadtxt(gt_instance_id_path, dtype='int')
    assert gt_mask.shape[0] == gt_instance_id.shape[0]
    
    labels = np.zeros(gt_mask.shape[1])
    for instance_id in range(gt_mask.shape[1]):
        mask_points = gt_mask[:, instance_id].bool()
        instance_label = gt_instance_id[mask_points][0]
        semantics_label = instance_label // 1000
        labels[instance_id] = semantics_label
    scene_name = os.path.splitext(os.path.basename(gt_mask_path))[0]
    split = train_test_dict[scene_name]
    output_path = f"../../dataset/data/replica_features/{split}/{scene_name}_labels"
    np.save(output_path, labels)

## generate instance feature (openscene features and ground truth masks)

In [60]:
# ADJUST PATH!!!
openscene_feature_path = glob.glob("../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/train/*_features.npy")
openscene_feature_path.extend(glob.glob("../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/val/*_features.npy"))
openscene_feature_path.extend(glob.glob("../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/test/*_features.npy"))

for feature_path in openscene_feature_path:
    print(feature_path)
    scene_name = os.path.splitext(os.path.basename(feature_path))[0]
    scene_name = scene_name.replace("_features", "")
    gt_mask_path = f"../../dataset/OpenYOLO3D/output/replica/replica_ground_truth_masks/{scene_name}.pt"

    point_features = np.load(feature_path)
    gt_mask = torch.load(gt_mask_path)[0]
    instance_features = np.zeros((gt_mask.shape[1], point_features.shape[1]))
    for instance_id in range(gt_mask.shape[1]):
        mask_points = gt_mask[:, instance_id].bool()
        mask_features = point_features[mask_points, :]
        instance_feature = np.mean(mask_features, axis=0)  # aggregate
        instance_features[instance_id, :] = instance_feature
    split = train_test_dict[scene_name]
    output_path = f"../../dataset/data/replica_features/{split}/{scene_name}_features"
    np.save(output_path, instance_features)
    

../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/train/office1_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/train/room0_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/train/office0_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/train/office2_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/val/room1_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/val/office3_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/test/office4_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/test/room2_features.npy


## Prompt Learning

In [1]:
# %%bash
# bash scripts/maple_prompt_scene/base2new_train_maple_prompt_scene.sh replica 1 ../../dataset/data/

Run this job and save the output to output/base2new/train_base/replica/shots_16/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx/seed1
Error while terminating subprocess (pid=2879446): 


In [3]:
# %%bash
# bash scripts/maple_prompt_scene/base2new_test_maple_prompt_scene.sh replica 1 ../../dataset/data/

Evaluating model
Runing the first phase job and save the output to output/base2new/test_new/replica/shots_16/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx/seed1


/cluster/54/nanriayaka/ml3d/models/multimodal-prompt-learning/clip/clip.py:23: UserWarning: PyTorch version 1.7.1 or higher is recommended
  warnings.warn("PyTorch version 1.7.1 or higher is recommended")


Error while terminating subprocess (pid=2879966): 


In [63]:
os.makedirs("../../dataset/data/replica_features/split_fewshot", exist_ok=True)

In [ ]:
"a {label} in a scene"

In [10]:
%%bash
bash scripts/maple_prompt_scene/xd_train_maple_prompt_scene.sh replica 1 ../../dataset/data/ vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets

Run this job and save the output to output/replica/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets_-1shots/seed1


/cluster/54/nanriayaka/ml3d/models/multimodal-prompt-learning/clip/clip.py:23: UserWarning: PyTorch version 1.7.1 or higher is recommended
  warnings.warn("PyTorch version 1.7.1 or higher is recommended")


Setting fixed seed: 1
***************
** Arguments **
***************
backbone: 
config_file: configs/trainers/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets.yaml
dataset_config_file: configs/datasets/replica.yaml
eval_only: False
head: 
load_epoch: None
model_dir: 
no_train: False
opts: ['DATASET.NUM_SHOTS', '-1']
output_dir: output/replica/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets_-1shots/seed1
resume: 
root: ../../dataset/data/
seed: 1
source_domains: None
target_domains: None
trainer: MaPLePromptScene
transforms: None
************
** Config **
************
DATALOADER:
  K_TRANSFORMS: 1
  NUM_WORKERS: 8
  RETURN_IMG0: False
  TEST:
    BATCH_SIZE: 100
    SAMPLER: SequentialSampler
  TRAIN_U:
    BATCH_SIZE: 32
    N_DOMAIN: 0
    N_INS: 16
    SAME_AS_X: True
    SAMPLER: RandomSampler
  TRAIN_X:
    BATCH_SIZE: 4
    N_DOMAIN: 0
    N_INS: 16
    SAMPLER: RandomSampler
DATASET:
  ALL_AS_UNLABELED: False
  CIFAR_C_LEVEL: 1
  CIFAR_C_

val: [<datasets.replica.Datum_feature object at 0x7f6def123760>, <datasets.replica.Datum_feature object at 0x7f6def122dd0>, <datasets.replica.Datum_feature object at 0x7f6def123790>, <datasets.replica.Datum_feature object at 0x7f6def123880>, <datasets.replica.Datum_feature object at 0x7f6def1239a0>, <datasets.replica.Datum_feature object at 0x7f6def1239d0>, <datasets.replica.Datum_feature object at 0x7f6def123a00>, <datasets.replica.Datum_feature object at 0x7f6def123a30>, <datasets.replica.Datum_feature object at 0x7f6def1238b0>, <datasets.replica.Datum_feature object at 0x7f6def123910>, <datasets.replica.Datum_feature object at 0x7f6def123940>, <datasets.replica.Datum_feature object at 0x7f6def123ac0>, <datasets.replica.Datum_feature object at 0x7f6def123fd0>, <datasets.replica.Datum_feature object at 0x7f6def123dc0>, <datasets.replica.Datum_feature object at 0x7f6def123f40>, <datasets.replica.Datum_feature object at 0x7f6def123c40>, <datasets.replica.Datum_feature object at 0x7f6def

-----------------------------------------------------------
-----------------------------------------------------------
train: [<datasets.replica.Datum_feature object at 0x7f6def122b60>, <datasets.replica.Datum_feature object at 0x7f6def1225f0>, <datasets.replica.Datum_feature object at 0x7f6def122920>, <datasets.replica.Datum_feature object at 0x7f6def121fc0>, <datasets.replica.Datum_feature object at 0x7f6def122a10>, <datasets.replica.Datum_feature object at 0x7f6def1227a0>, <datasets.replica.Datum_feature object at 0x7f6def122980>, <datasets.replica.Datum_feature object at 0x7f6def122950>, <datasets.replica.Datum_feature object at 0x7f6def122350>, <datasets.replica.Datum_feature object at 0x7f6def122bc0>, <datasets.replica.Datum_feature object at 0x7f6def1229e0>, <datasets.replica.Datum_feature object at 0x7f6def122230>, <datasets.replica.Datum_feature object at 0x7f6def1224a0>, <datasets.replica.Datum_feature object at 0x7f6def1222c0>, <datasets.replica.Datum_feature object at 0x7f

...

=> result
* total: 106
* correct: 46
* accuracy: 43.4%
* error: 56.6%
* macro_f1: 24.8%
epoch [200/200] batch [20/44] time 0.039 (0.049) data 0.000 (0.009) loss 0.7964 (0.8385) lr 8.6352e-07 eta 0:00:01
epoch [200/200] batch [40/44] time 0.040 (0.044) data 0.000 (0.005) loss 0.1755 (0.8252) lr 8.6352e-07 eta 0:00:00
Evaluate on the *val* set


100%|██████████| 2/2 [00:00<00:00,  8.10it/s]


=> result
* total: 106
* correct: 46
* accuracy: 43.4%
* error: 56.6%
* macro_f1: 24.8%
Checkpoint saved to output/replica/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets_-1shots/seed1/MultiModalPromptSceneLearner/model.pth.tar-200
Finish training
Deploy the model with the best val performance
Loading weights to MultiModalPromptSceneLearner from "output/replica/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets_-1shots/seed1/MultiModalPromptSceneLearner/model-best.pth.tar" (epoch = 137)
Evaluate on the *test* set


100%|██████████| 1/1 [00:00<00:00,  4.49it/s]


=> result
* total: 83
* correct: 47
* accuracy: 56.6%
* error: 43.4%
* macro_f1: 30.3%
Elapsed: 0:07:45


In [22]:
!rm -r output/evaluation/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets_-1shots/replica/seed1

In [23]:
%%bash
bash scripts/maple_prompt_scene/xd_test_maple_prompt_scene.sh replica 1 ../../dataset/data/ vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets

Run this job and save the output to output/evaluation/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets_-1shots/replica/seed1


/cluster/54/nanriayaka/ml3d/models/multimodal-prompt-learning/clip/clip.py:23: UserWarning: PyTorch version 1.7.1 or higher is recommended
  warnings.warn("PyTorch version 1.7.1 or higher is recommended")


Setting fixed seed: 1
***************
** Arguments **
***************
backbone: 
config_file: configs/trainers/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets.yaml
dataset_config_file: configs/datasets/replica.yaml
eval_only: True
head: 
load_epoch: None
model_dir: output/replica/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets_-1shots/seed1
no_train: False
opts: []
output_dir: output/evaluation/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets_-1shots/replica/seed1
resume: 
root: ../../dataset/data/
seed: 1
source_domains: None
target_domains: None
trainer: MaPLePromptScene
transforms: None
************
** Config **
************
DATALOADER:
  K_TRANSFORMS: 1
  NUM_WORKERS: 8
  RETURN_IMG0: False
  TEST:
    BATCH_SIZE: 100
    SAMPLER: SequentialSampler
  TRAIN_U:
    BATCH_SIZE: 32
    N_DOMAIN: 0
    N_INS: 16
    SAME_AS_X: True
    SAMPLER: RandomSampler
  TRAIN_X:
    BATCH_SIZE: 4
    N_DOMAIN: 0
    N_INS: 16
    SAMPL

val: [<datasets.replica.Datum_feature object at 0x7f76085de170>, <datasets.replica.Datum_feature object at 0x7f76085dc280>, <datasets.replica.Datum_feature object at 0x7f76085de1a0>, <datasets.replica.Datum_feature object at 0x7f76085dea10>, <datasets.replica.Datum_feature object at 0x7f76085de320>, <datasets.replica.Datum_feature object at 0x7f76085de350>, <datasets.replica.Datum_feature object at 0x7f76085de3b0>, <datasets.replica.Datum_feature object at 0x7f76085de3e0>, <datasets.replica.Datum_feature object at 0x7f76085de9e0>, <datasets.replica.Datum_feature object at 0x7f76085de290>, <datasets.replica.Datum_feature object at 0x7f76085de2c0>, <datasets.replica.Datum_feature object at 0x7f76085de410>, <datasets.replica.Datum_feature object at 0x7f76085dea40>, <datasets.replica.Datum_feature object at 0x7f76085de7d0>, <datasets.replica.Datum_feature object at 0x7f76085de950>, <datasets.replica.Datum_feature object at 0x7f76085de650>, <datasets.replica.Datum_feature object at 0x7f7608

-----------------------------------------------------------
-----------------------------------------------------------
train: [<datasets.replica.Datum_feature object at 0x7f76085dcbb0>, <datasets.replica.Datum_feature object at 0x7f76085dcac0>, <datasets.replica.Datum_feature object at 0x7f76085dd810>, <datasets.replica.Datum_feature object at 0x7f76085dcd60>, <datasets.replica.Datum_feature object at 0x7f76085dd660>, <datasets.replica.Datum_feature object at 0x7f76085dd780>, <datasets.replica.Datum_feature object at 0x7f76085dd540>, <datasets.replica.Datum_feature object at 0x7f76085dcfa0>, <datasets.replica.Datum_feature object at 0x7f76085dd270>, <datasets.replica.Datum_feature object at 0x7f76085dc160>, <datasets.replica.Datum_feature object at 0x7f76085dc910>, <datasets.replica.Datum_feature object at 0x7f76085dd6c0>, <datasets.replica.Datum_feature object at 0x7f76085dca90>, <datasets.replica.Datum_feature object at 0x7f76085dd2d0>, <datasets.replica.Datum_feature object at 0x7f

val: [<datasets.replica.Datum_feature object at 0x7f76085de170>, <datasets.replica.Datum_feature object at 0x7f76085dc280>, <datasets.replica.Datum_feature object at 0x7f76085de1a0>, <datasets.replica.Datum_feature object at 0x7f76085dea10>, <datasets.replica.Datum_feature object at 0x7f76085de320>, <datasets.replica.Datum_feature object at 0x7f76085de350>, <datasets.replica.Datum_feature object at 0x7f76085de3b0>, <datasets.replica.Datum_feature object at 0x7f76085de3e0>, <datasets.replica.Datum_feature object at 0x7f76085de9e0>, <datasets.replica.Datum_feature object at 0x7f76085de290>, <datasets.replica.Datum_feature object at 0x7f76085de2c0>, <datasets.replica.Datum_feature object at 0x7f76085de410>, <datasets.replica.Datum_feature object at 0x7f76085dea40>, <datasets.replica.Datum_feature object at 0x7f76085de7d0>, <datasets.replica.Datum_feature object at 0x7f76085de950>, <datasets.replica.Datum_feature object at 0x7f76085de650>, <datasets.replica.Datum_feature object at 0x7f7608

-----------------------------------------------------------
Label range: 0-47
Unique labels: {0, 3, 4, 5, 6, 7, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 30, 31, 32, 35, 36, 37, 38, 39, 40, 41, 43, 44, 45, 46, 47}
Label range: 0-47
Unique labels: {0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 16, 17, 19, 20, 21, 22, 24, 25, 26, 28, 29, 31, 35, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47}
Label range: 0-0
Unique labels: {0}
Building transform_train
+ random resized crop (size=(224, 224), scale=(0.08, 1.0))
+ random flip
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
Building transform_test
+ resize the smaller edge to 224
+ 224x224 center crop
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
---------  -------
Dataset    Replica
# classes  48
# train_x  178
# val      106
# test     192
---------  

100%|██████████| 2/2 [00:00<00:00,  2.23it/s]


=> result
* total: 192
* correct: 0
* accuracy: 0.0%
* error: 100.0%
* macro_f1: 0.0%
